In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [4]:
i = -0.4
j = -0.1
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})"
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}}


100%|██████████| 2/2 [00:01<00:00,  1.63it/s]


{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}}


100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


In [5]:
save_model=True
print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf_only_on_test = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    base_model = model_utils.get_model(max_iter=500)
    experiment.clf_only_on_test = copy.deepcopy(base_model)
    experiment.clf_only_on_test.fit(experiment.X_test, experiment.y_te_onehot)

    if save_model:
        model_utils.save_model(experiment.clf_only_on_test, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')

Training classifier for experiment: Census19_correlation_0_subgroup_col_name_SEX_marginal_prior_1_corr_btn_sens_and_output_per_subgroup_LPAREN-0.4, -0.1RPAREN
Loaded classifier for experiment from file: Census19_correlation_0_subgroup_col_name_SEX_marginal_prior_1_corr_btn_sens_and_output_per_subgroup_LPAREN-0.4, -0.1RPAREN


In [26]:
hidden_layer_size_tuples = [(64, 32, 16, 8), (32, 16, 8), (16, 8)]
experiment.clfs={}

for i, hidden_layer_size_tuple in enumerate(hidden_layer_size_tuples):
    base_model = model_utils.get_model(max_iter=500, hidden_layer_sizes=hidden_layer_size_tuple)
    clf = copy.deepcopy(base_model)
    clf.fit(experiment.X_test, experiment.y_te_onehot)
    experiment.clfs[hidden_layer_size_tuple] = clf
    

In [43]:
save_model = True

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42, max_depth=7)
clf.fit(experiment.X_test, experiment.y_te_onehot)
experiment.clfs['DT'] = clf

In [40]:
clf.predict_proba(experiment.X_train)

[array([[0.7474842 , 0.2525158 ],
        [0.27403532, 0.72596468],
        [0.4626506 , 0.5373494 ],
        ...,
        [0.40529189, 0.59470811],
        [0.40529189, 0.59470811],
        [0.81881051, 0.18118949]]),
 array([[0.2525158 , 0.7474842 ],
        [0.72596468, 0.27403532],
        [0.5373494 , 0.4626506 ],
        ...,
        [0.59470811, 0.40529189],
        [0.59470811, 0.40529189],
        [0.18118949, 0.81881051]])]

In [94]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==0].copy().reset_index(drop=True).drop(['is_train'], axis=1)

p = -0.1
x = original_test_df[original_test_df['SEX']==0][['MAR', 'PINCP']].value_counts().to_numpy().min()
n = (x * 4) // (1 + p)

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p1=-0.1, p2=-0.1, n=2*n, subgroup_col_name='SEX', transformed_already=True, return_indices_only=True)

experiment.X_test_balanced_corr = experiment.X_test.loc[temp_indices].reset_index(drop=True)
experiment.y_te_balanced_corr = experiment.y_te[temp_indices]
experiment.y_te_onehot_balanced_corr = experiment.y_te_onehot[temp_indices]

[0, 1]
[16666.0, 16666.0]
{0: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}, 1: {(0, 1): 4583, (0, 0): 3749, (1, 1): 3750.0, (1, 0): 4584.0}}


100%|██████████| 2/2 [00:00<00:00, 30.48it/s]


In [33]:
hidden_layer_size_tuples = [(64, 32, 16, 8), (32, 16, 8), (16, 8)]
experiment.clfs_balanced_corr={}

for i, hidden_layer_size_tuple in enumerate(hidden_layer_size_tuples):
    base_model = model_utils.get_model(max_iter=500, hidden_layer_sizes=hidden_layer_size_tuple)
    experiment.clfs_balanced_corr[hidden_layer_size_tuple] = copy.deepcopy(base_model)
    experiment.clfs_balanced_corr[hidden_layer_size_tuple].fit(experiment.X_test_balanced_corr, experiment.y_te_onehot_balanced_corr)

In [45]:
clf = DecisionTreeClassifier(random_state=42, max_depth=7)
clf.fit(experiment.X_test_balanced_corr, experiment.y_te_onehot_balanced_corr)
experiment.clfs_balanced_corr['DT'] = clf

In [98]:
def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'SEX', indices=None):
    if isinstance(clf, DecisionTreeClassifier):
        y_tr_pred = np.argmax(clf.predict_proba(X_train)[1], axis=1)
    else:
        y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    subgroup_vals_tr = X_train[f'{subgroup_col_name}_1'].to_numpy().ravel().astype(int)
    subgroup_vals_te = X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    sens_pred_LOMIA = LOMIA_attack(experiment, clf, X_test, y_te, experiment.ds.ds.meta, indices=indices)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())
    correct_indices_LOMIA = (sens_pred_LOMIA == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

    male_indices = np.where(subgroup_vals_tr==0)[0]
    female_indices = np.where(subgroup_vals_tr==1)[0]
    print(correct_indices_LOMIA[np.where(subgroup_vals_te==0)[0]].mean())
    print(correct_indices_LOMIA[np.where(subgroup_vals_te==1)[0]].mean())
    perf_dict = {
        'ASRD_CSMIA': round(100 * np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'ASRD_LOMIA': round(100 * np.ptp([correct_indices_LOMIA[subgroup_vals_te==i].mean() for i in [0, 1]]), 2),
        'EOD': round(equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'DPD': round(demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'MA_Male': 100 * accuracy_score(y_tr.ravel()[male_indices], y_tr_pred[male_indices]),
        'MA_Female': 100 * accuracy_score(y_tr.ravel()[female_indices], y_tr_pred[female_indices])
    }

    return perf_dict


In [101]:
model_arch = hidden_layer_size_tuples[1]
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr),
    'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices)
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

0.69872
0.549
0.5272410896435857
0.5631825273010921


In [111]:
experiment.X_test_balanced_corr.shape[0]/experiment.X_test.shape[0]

0.66664

In [102]:
res_dict_df

,ASRD_CSMIA,ASRD_LOMIA,EOD,DPD,MA_Male,MA_Female
w/o BCorr,12.52,14.97,0.0674,0.0178,73.208,78.16
w Bcorr,2.06,3.59,0.0416,0.0015,73.888,77.90


In [103]:
print(res_dict_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & ASRD_CSMIA & ASRD_LOMIA & EOD & DPD & MA_Male & MA_Female \\
\midrule
w/o BCorr & 12.520000 & 14.970000 & 0.067400 & 0.017800 & 73.208000 & 78.160000 \\
w Bcorr & 2.060000 & 3.590000 & 0.041600 & 0.001500 & 73.888000 & 77.900000 \\
\bottomrule
\end{tabular}



In [64]:
res_dict

{'w/o BCorr': {'MA_Female': 78.16,
  'MA_Male': 73.208,
  'EOD': 0.06735999999999998,
  'DPD': 0.017839999999999967,
  'ASRD': 0.12524000000000002},
 'w Bcorr': {'MA_Female': 77.9,
  'MA_Male': 73.888,
  'EOD': 0.04160000000000008,
  'DPD': 0.0014800000000000368,
  'ASRD': 0.020580823232929313}}

In [106]:
for model_arch in experiment.clfs:
    print(model_arch)

(64, 32, 16, 8)
(32, 16, 8)
(16, 8)
DT


In [61]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr))

{'MA_Female': 77.164, 'MA_Male': 72.712, 'EOD': 0.057999999999999996, 'DPD': 0.013480000000000047, 'ASRD': 0.08372000000000002}
{'MA_Female': 78.16, 'MA_Male': 73.208, 'EOD': 0.06735999999999998, 'DPD': 0.017839999999999967, 'ASRD': 0.12524000000000002}
{'MA_Female': 78.99199999999999, 'MA_Male': 73.628, 'EOD': 0.06672, 'DPD': 0.013080000000000092, 'ASRD': 0.14991999999999994}
{'MA_Female': 75.932, 'MA_Male': 72.348, 'EOD': 0.03735999999999995, 'DPD': 0.0015199999999999658, 'ASRD': 0.05384}


In [110]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices))

0.5234609384375375
0.5688227529101164
{'ASRD_CSMIA': 1.37, 'ASRD_LOMIA': 4.54, 'EOD': 0.0493, 'DPD': 0.0047, 'MA_Male': 72.272, 'MA_Female': 76.728}
0.5272410896435857
0.5631825273010921
{'ASRD_CSMIA': 2.06, 'ASRD_LOMIA': 3.59, 'EOD': 0.0416, 'DPD': 0.0015, 'MA_Male': 73.888, 'MA_Female': 77.9}
0.4674186967478699
0.49795991839673587
{'ASRD_CSMIA': 0.46, 'ASRD_LOMIA': 3.05, 'EOD': 0.0494, 'DPD': 0.0172, 'MA_Male': 75.312, 'MA_Female': 78.532}
0.4499579983199328
0.4499579983199328
{'ASRD_CSMIA': 0.46, 'ASRD_LOMIA': 0.0, 'EOD': 0.0353, 'DPD': 0.003, 'MA_Male': 72.38799999999999, 'MA_Female': 75.62}


In [104]:
get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, {'SEX': 0})

0.10008400408079707